## Requirements

In [54]:
%pip install youtube-transcript-api openai pyonepassword sqlite_utils

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Youtube transcript summarizer

In [55]:
video_id = "Wdnwt0COZGM" # ID from video URL

In [56]:
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from pyonepassword import OP

op = OP()
openai_key = op.item_get('ociiuq6gchlgrjr2fopzbndrnq').credential
client = OpenAI(api_key=openai_key)

transcript = YouTubeTranscriptApi.get_transcript(video_id) 
full_transcript = ' '.join(entry['text'] for entry in transcript) # Transcript is retuned in chunks for some reason

model = "gpt-4o"
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        { 
            "role": "system", 
            "content": "You summarize YouTube transcripts to help traders determine a directional bias and have a profitable day." 
        },
        {
            "role": "user",
            "content": (
                "Summarize the transcript and determine Tino's directional bias. "
                "Also read between the lines to determine what's actually being said and explain. "
                "Use markdown formatting. "
                "Write a personal update about Tino at the end if he mentions anything.\n\n"
                f"Transcript: {full_transcript}"
            ),
        }
    ]
)
ai_explanation = chat_completion.choices[0].message.content

from IPython.display import display, Markdown
display(Markdown(ai_explanation))

### Summary
Tino is sharing his insights on a challenging trading day with significant market movements. He highlights the unpredictability of trading, using his break-even trade on the NASDAQ as an example. Tino discusses market jitters due to geopolitical tensions involving Israel, affecting investor sentiment. He notes a mix of reactions in the stock market, with Amazon doing well while others like Apple have not. Tino is also concerned about possible impacts from upcoming economic data, specifically the U.S. non-farm payrolls. He explores the threat of inflation and interest rates in regard to the bond market. 

### Tino's Directional Bias
- **Bitcoin**: He notes potential down movements but is holding a cautious view, especially with the non-farm payrolls data pending.
- **NASDAQ**: He exhibits a tentative bias towards a recovery after a drop, citing Amazon's strong earnings as a potential catalyst.
- **Gold**: Tino expects it to move upward amidst geopolitical tensions.
  
### Reading Between the Lines
- Tino is cautious and looks to evaluate the broader implications of current events like geopolitical tensions and economic indicators on market movements.
- He acknowledges market volatility influenced by bigger players, such as ETFs, and external factors like geopolitical events.
- Tino is skeptical about long-term bullish scenarios without a substantial market correction or crash.

### Personal Update
Tino recently returned from Cyprus, where he implies he was not on vacation but working, as evident from his commitments to live streams and market analysis. He humorously references his lack of costumes for Halloween, leaning into a self-deprecating joke about his own appearance being scary enough. There was no significant personal development mentioned beyond his current activities related to the market and trading activities.

## Database

In [57]:
from sqlite_utils import Database
from datetime import datetime

# Initialize the database
db = Database("video_data.db")
table = "video_analysis"

# Check for duplicate video_id
if not db[table].exists() or not any(row['video_id'] == video_id for row in db[table].rows):
    # Insert a new row into the video_analysis table
    db[table].insert({
        "datetime": datetime.now(),
        "video_id": video_id,
        "transcript": full_transcript,
        "model": model,
        "ai_explanation": ai_explanation
    })
else:
    print(f"Video ID {video_id} already exists in the database.")

Video ID Wdnwt0COZGM already exists in the database.


## Possible future work:

To enhance the morning summary for a day trader focusing on cryptocurrency, you can incorporate additional data points that provide a comprehensive view of the market. Here are some suggestions:

1. **Market Sentiment Analysis**:
   - Use sentiment analysis tools to gauge the overall market sentiment from social media platforms like Twitter, Reddit, and crypto forums.
   - Incorporate sentiment scores or trends to provide insights into the general mood of the market.

2. **Technical Indicators**:
   - Include key technical indicators such as moving averages (e.g., 50-day, 200-day), RSI (Relative Strength Index), MACD (Moving Average Convergence Divergence), and Bollinger Bands.
   - Highlight any significant technical patterns or signals (e.g., golden cross, death cross).
   - Ben Cowan (into the crypto verso): bull market support band, dominance

3. **News and Events**:
   - Summarize recent news articles or headlines that could impact the crypto market, such as regulatory changes, major partnerships, or technological advancements.
   - Include a calendar of upcoming events like earnings reports, economic data releases, or major conferences.

4. **On-Chain Data**:
   - Provide insights from on-chain data such as transaction volumes, active addresses, and large transactions (whale movements).
   - Highlight any unusual activity that could indicate market manipulation or significant interest.

5. **Volatility Index**:
   - Include a measure of market volatility, such as the Crypto Volatility Index (CVI), to help traders assess potential risk levels.

6. **Exchange Flows**:
   - Track the inflow and outflow of cryptocurrencies on major exchanges to identify potential buying or selling pressure.

7. **Correlation with Other Markets**:
   - Analyze the correlation between cryptocurrencies and other asset classes like stocks, gold, or fiat currencies to understand broader market trends.

8. **Regulatory Updates**:
   - Provide updates on any regulatory developments that could affect the crypto market, such as new laws or enforcement actions.

9. **Macro-Economic Indicators**:
   - Include relevant macroeconomic indicators such as interest rates, inflation data, and employment figures that could influence market sentiment.

10. **Community and Developer Activity**:
    - Track activity on platforms like GitHub to gauge the level of development and community engagement for specific cryptocurrencies.

By incorporating these data points, you can provide a more holistic and actionable morning summary for day traders in the crypto market.

## Experiments

In [58]:
%pip install tradingview-ta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
from tradingview_ta import TA_Handler, Interval, Exchange

btc = TA_Handler(
    symbol="BTCUSDT.P",
    screener="crypto",
    exchange="binance",
    interval=Interval.INTERVAL_1_DAY
)
print("BTCUSDT Futures", btc.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

BTCUSDT Futures {'RECOMMENDATION': 'BUY', 'BUY': 15, 'SELL': 1, 'NEUTRAL': 10}
